In [1]:
import pandas as pd
import numpy as np

# Load data
df = pd.read_csv('../dataset/dataset.csv')

# --------------------------------------------------
# 1.1 Signal Decoding
# --------------------------------------------------
def parse_signal(signal_str):
    cleaned = signal_str.strip('[]').replace(' ', '')
    parts = cleaned.split('),(')
    complex_samples = []
    for p in parts:
        p = p.replace('(', '').replace(')', '')
        try:
            # Convert directly to complex number
            complex_samples.append(complex(p))
        except ValueError:
            # Skip any invalid entries
            continue
    return np.array(complex_samples[:208])

# Apply to all rows
X_signal = np.array([parse_signal(s) for s in df['received_signal']])
# print(X_signal[:1])
X_signal = np.hstack([X_signal.real, X_signal.imag])  # (15000, 416)

# --------------------------------------------------
# 1.2 Secret Code Handling
# --------------------------------------------------
def parse_secret_code(code_str):
    return np.array([int(x) for x in code_str.strip('[]').split(', ')])

X_secret = np.array([parse_secret_code(c) for c in df['secret_code']])  # (15000, 13)

# --------------------------------------------------
# 1.3 Target Preparation
# --------------------------------------------------
y = df[['jet1_x', 'jet1_y', 'jet1_z', 
        'jet2_x', 'jet2_y', 'jet2_z']].values  # (15000, 6)

In [2]:
print(X_signal[:1])

[[ 0.11931579  0.85771666 -1.33474694 -0.50391545 -2.29890648 -2.36311255
  -1.73337179 -2.09075436 -1.68077154 -2.06851403 -2.23007812 -1.68373539
  -0.99208445 -1.52579929 -2.13198495 -1.542111   -1.572531   -1.17894242
  -0.43579253 -0.37435658 -0.20439189 -0.34568808 -0.3036721   0.78714854
  -0.13114397  0.91876952  0.59695031  0.46554019  0.30731446  0.57871762
   1.3063269   1.32965779  1.84256292  2.62761689  3.8078487   2.76445951
   2.48637366  2.88149825  2.38199167  2.72596637  1.39522053  0.8044313
   0.23753253 -0.8312159  -2.08004069 -2.24298221 -2.99989836 -3.15122902
  -3.34715525 -3.41449118 -1.9398927  -2.12972652 -1.46899167 -0.57031672
  -0.59802137 -0.68256009  0.22612962  0.72654863 -0.07467601 -0.28339311
  -1.24302304 -0.46743044 -1.8572562  -2.77289709 -3.06969035 -3.11564192
  -3.84457788 -3.57200481 -3.62208702 -3.10760311 -2.81848032 -1.19532471
  -0.52973206  1.028434    2.24372655  3.21794968  3.74768431  4.09193022
   3.67424067  1.20070218  0.9215773  -

In [3]:
X_signal

array([[ 0.11931579,  0.85771666, -1.33474694, ...,  1.35653706,
         0.09700217,  0.68696531],
       [-0.035028  , -0.9321786 , -1.07103858, ..., -0.31218129,
        -0.86626219, -1.67845885],
       [-0.01784629, -0.30454525, -0.63100547, ...,  2.48600477,
         2.57460476,  2.38237365],
       ...,
       [ 0.18681767,  0.2127872 , -0.10902881, ..., -1.86084518,
        -2.68116411, -3.5353162 ],
       [ 0.28481301,  1.10820823,  1.38517485, ...,  1.33907262,
         1.06573864,  0.82789565],
       [ 0.19803341,  0.78227132,  0.95982124, ...,  1.25057382,
         1.13393457,  1.30820358]])

In [4]:
X_signal.shape

(15000, 416)

In [5]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

# --------------------------------------------------
# 2.1 I/Q Signal Engineering (CNN Input)
# --------------------------------------------------
# Normalize signal features column-wise
scaler_signal = StandardScaler()
X_signal_normalized = scaler_signal.fit_transform(X_signal)  # (15000, 416)

# Reshape for CNN (samples, timesteps, channels)
# Assuming original signal had 208 complex samples → 208 timesteps × 2 channels (real/imag)
X_signal_reshaped = X_signal_normalized.reshape(-1, 208, 2)  # (15000, 208, 2)

# --------------------------------------------------
# 2.2 Secret Code Normalization (Dense Input)
# --------------------------------------------------
scaler_secret = StandardScaler()
X_secret_normalized = scaler_secret.fit_transform(X_secret)  # (15000, 13)

# --------------------------------------------------
# 2.3 Target Normalization
# --------------------------------------------------
scaler_target = StandardScaler()
y_normalized = scaler_target.fit_transform(y)  # (15000, 6)

# --------------------------------------------------
# Data Splitting (Keep Branches Separate)
# --------------------------------------------------
# Split signal and secret code data separately but consistently
# First split: 80% train, 20% temp
X_sig_train, X_sig_temp, X_sec_train, X_sec_temp, y_train, y_temp = train_test_split(
    X_signal_reshaped,  # (15000, 208, 2)
    X_secret_normalized,  # (15000, 13)
    y_normalized,  # (15000, 6)
    test_size=0.2, 
    random_state=42
)

# Second split: 50% validation, 50% test (of the 20% temp)
X_sig_val, X_sig_test, X_sec_val, X_sec_test, y_val, y_test = train_test_split(
    X_sig_temp, 
    X_sec_temp, 
    y_temp, 
    test_size=0.5, 
    random_state=42
)

print("Training shapes:", X_sig_train.shape, X_sec_train.shape, y_train.shape)
print("Validation shapes:", X_sig_val.shape, X_sec_val.shape, y_val.shape)
print("Test shapes:", X_sig_test.shape, X_sec_test.shape, y_test.shape)

Training shapes: (12000, 208, 2) (12000, 13) (12000, 6)
Validation shapes: (1500, 208, 2) (1500, 13) (1500, 6)
Test shapes: (1500, 208, 2) (1500, 13) (1500, 6)


In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv1D, MaxPooling1D, Flatten, Dense, concatenate
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

# --------------------------------------------------
# 3.1 Dual Input Branches
# --------------------------------------------------
# Branch 1: Radar Signal Processor (CNN)
signal_input = Input(shape=(208, 2), name="radar_signal")  # (timesteps, channels)
x = Conv1D(32, 3, activation="relu", padding="same")(signal_input)
x = MaxPooling1D(2)(x)  # Output: (104, 32)
x = Conv1D(64, 3, activation="relu", padding="same")(x)
x = MaxPooling1D(2)(x)  # Output: (52, 64)
x = Flatten()(x)         # Output: (52*64 = 3328)
x = Dense(128, activation="relu")(x)  # Compress to 128 features

# Branch 2: Code Breaker (Dense Network)
code_input = Input(shape=(13,), name="secret_code")
y = Dense(64, activation="relu")(code_input)
y = Dense(32, activation="relu")(y)

# --------------------------------------------------
# 3.2 Combined Strike Force
# --------------------------------------------------
merged = concatenate([x, y], name="intel_fusion")  # 128 + 32 = 160 features
z = Dense(64, activation="relu")(merged)
z = Dense(32, activation="relu")(z)
outputs = Dense(6, activation="linear")(z)  # 6 coordinates

# --------------------------------------------------
# War Machine Assembly
# --------------------------------------------------
model = Model(inputs=[signal_input, code_input], outputs=outputs)

# --------------------------------------------------
# Weapon Calibration (Compilation)
# --------------------------------------------------
model.compile(
    optimizer=Adam(learning_rate=0.001),
    loss="mse",  # Mean Squared Error for coordinates
    metrics=["mae"]  # Monitor Mean Absolute Error
)

# --------------------------------------------------
# Reconnaissance Report (Model Summary)
# --------------------------------------------------
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 radar_signal (InputLayer)   [(None, 208, 2)]             0         []                            
                                                                                                  
 conv1d (Conv1D)             (None, 208, 32)              224       ['radar_signal[0][0]']        
                                                                                                  
 max_pooling1d (MaxPooling1  (None, 104, 32)              0         ['conv1d[0][0]']              
 D)                                                                                               
                                                                                                  
 conv1d_1 (Conv1D)           (None, 104, 64)              6208      ['max_pooling1d[0][0]']   

1. Why ReLU in Hidden Layers?
ReLU (Rectified Linear Unit) is the default activation for hidden layers

2. Linear Activation in the Final Layer?
The final layer (Dense(6, activation="linear") uses linear activation because:

Regression Task:
You’re predicting 6 continuous coordinates (like x, y, z positions). A linear activation outputs unbounded values, which is ideal for regression.


### 1. CNN Branch: The Tea Brewing Process  
**Input Signal** = A pot of tea with:  
- **208 tea leaves** (timesteps)  
- **2 types of leaves** (channels: e.g., *sweetness* and *bitterness*).  

#### Step 1: First Strainer (`Conv1D` with 32 filters)  
- You use **32 unique strainers** (filters) to pour the tea through.  
- Each strainer has a **window size of 3 leaves** (`kernel_size=3`) and mixes both leaf types (channels).  
- **Output**: 32 distinct "flavors" of tea (feature maps), e.g., bitterness spikes.  

#### Step 2: Concentrate the Tea (`MaxPooling1D`)  
- Boil down the tea to **half its volume** (104 timesteps), keeping only the strongest flavors (max values).  

#### Step 3: Second Strainer (`Conv1D` with 64 filters)  
- Use **64 finer strainers** to detect complex flavor combinations (e.g., *bitter-sweet oscillations*).  
- **Output**: 64 refined teas.  
- **Concentrate again** (`MaxPooling1D` → 52 timesteps).  

#### Step 4: Flatten & Compress (`Dense` Layer)  
- **Flatten**: Pour all 64 teas into a single stream (3328 features).  
- **`Dense(128)`**: Reduce to **128 essential flavors** (critical for mixing with the code branch).  

---

### 2. Code Branch: Secret Spice Preparation  
**Input Code** = A **13-ingredient secret spice mix**.  
- **`Dense(64)`**: Extract 64 aromatic notes (e.g., *cinnamon, nutmeg*).  
- **`Dense(32)`**: Refine to **32 potent spices** (e.g., *smoky, earthy*).  

---

### 3. Fusion: Mixing Tea & Spices  
- **Concatenate**: Combine the **128 tea flavors** and **32 spices** → 160-dimensional "super-blend."  
- **`Dense(64 → 32)`**: Harmonize the mixture into a balanced brew.  
- **Output Layer (`Dense(6)`)**: Predict **6 coordinates** (e.g., perfect tea temperature, sweetness).  

---

### 🚨 What the Tea Analogy Misses  
1. **Learning Weights**:  
   - Filters (strainers) **adapt** during training (unlike static strainers).  

2. **Non-Linear Activation (`ReLU`)**:  
   - Acts like a "flavor enhancer," removing bland/negative tastes (values < 0).  

3. **Optimizer & Loss Function**:  
   - `Adam` = **master brewer** adjusting strainers/spices to minimize errors (`MSE` loss).  

4. **Concatenation of Branches**:  
   - Mixing radar signals (tea) + secret codes (spices) has no direct tea analogy.  

5. **Training Process**:  
   - Iterative refinement over trials (epochs) with `EarlyStopping` to avoid over-brewing (overfitting).  

---

### 🍵 Final Takeaway  
The tea analogy simplifies **hierarchical feature extraction** and **fusion**, but real neural networks add:  
- Adaptability (dynamic strainers),  
- Non-linear transformations (`ReLU`),  
- Mathematical optimization (`Adam`).  

The code is a **self-adjusting recipe**—not a fixed kitchen process! 🔮

## 1. The Two Spy Agencies (Dual Inputs)
## James Bond, Jason Bourne

## 2. How James Bond Works (CNN Operations)

### a. Conv1D Layer (32 filters, kernel_size=3)
- **Imagine Bond has 32 special glasses (filters)**
- **Each glass looks at 3 time-steps at a time** (kernel_size=3)
- **"Same" padding** = Adds invisible zeros at edges so Bond doesn't miss end signals
- **Relu activation** = "If you see something interesting, shout louder!"

### b. MaxPooling1D (2)
- **"Only remember the most important clue every 2 steps"**
- Reduces data from 208 → 104 steps
- **Repeat with bigger glasses:**
  - Now 64 filters (better pattern recognition)
  - Reduces 104 → 52 steps

### c. Flatten
- Takes all clues (52 steps × 64 filters) = 3328 pieces of intel
- Puts them in one long list

### d. Dense(128)
- Condenses 3328 pieces → 128 most important clues

---

## 3. How Jason Bourne Works (Code Breaking)
**Secret Code Processing:**
1. `Dense(64)`: Turns 13 digits → 64 hidden patterns
2. `Dense(32)`: Condenses to 32 ultimate code secrets

---

## 4. The Team-Up (Concatenation)
- Combines Bond's 128 clues + Bourne's 32 secrets = **160 combined intel**
- Works together through final layers:
  - `Dense(64)`: Team discussion to align findings
  - `Dense(32)`: Final strategy meeting
  - `Output(6)`: 3D coordinates (x,y,z + maybe velocity?)

---

## 5. Why This Works?

### CNN Advantages for Radar:
- **Kernel** = Pattern detector (small time-window scanner)
- **Pooling** = "Don't get distracted by noise"
- Learns **hierarchical patterns**: small details → big picture

### Secret Code Handling:
- Dense layers find relationships between code digits
- Even if code is scrambled, network learns to decode

---

## 6. Training Setup
- **Adam Optimizer**: Smart learning rate adjustment
- **MSE Loss**: Punishes big coordinate errors more than small ones
- **MAE Metric**: Tracks average error distance

In [7]:
from tensorflow.keras.layers import Dropout

# --------------------------------------------------
# Revised Branch 1: Radar Signal Processor (with Dropout)
# --------------------------------------------------
signal_input = Input(shape=(208, 2))
x = Conv1D(32, 3, activation='relu')(signal_input)
x = MaxPooling1D(2)(x)
x = Conv1D(64, 3, activation='relu')(x)
x = MaxPooling1D(2)(x)
x = Flatten()(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.5)(x)  # 50% dropout to break enemy decoy patterns

# --------------------------------------------------
# Revised Branch 2: Code Breaker (with Dropout)
# --------------------------------------------------
code_input = Input(shape=(13,))
y = Dense(64, activation='relu')(code_input)
y = Dense(32, activation='relu')(y)
y = Dropout(0.3)(y)  # 30% dropout to counter code spoofing

# Merge and final layers remain unchanged

1. Radar Signal Branch (Tea Brewing with Dropout)
After extracting 128 essential flavors (Dense(128)):

Dropout(0.5): Before mixing with the code spices, randomly block 50% of the tea flavors (e.g., sweetness, bitterness) in each training batch.

Why? Forces the team to learn multiple ways to make good tea, even if half the flavors are missing.

2. Code Branch (Spice Preparation with Dropout)
After refining 32 spices (Dense(32)):

Dropout(0.3): Randomly remove 30% of the spices (e.g., cinnamon, nutmeg) in each batch.

Why? Ensures the recipe doesn’t depend too much on any single spice.

In [8]:
early_stop = EarlyStopping(
    monitor="val_loss", 
    patience=10,          # Abort if no improvement for 10 epochs
    restore_best_weights=True,  # Revert to best-known parameters
    verbose=1
)

In [9]:
test_loss, test_mae = model.evaluate(  
    [X_sig_test, X_sec_test],   
    y_test,  
    verbose=0  
)  
print(f"Test MSE: {test_loss:.4f} (Normalized)")  
print(f"Test MAE: {test_mae:.4f} (Normalized)")  

# Inverse-transform for real-world error  
y_pred_normalized = model.predict([X_sig_test, X_sec_test])  
y_pred_real = scaler_target.inverse_transform(y_pred_normalized)  
y_test_real = scaler_target.inverse_transform(y_test)  

# Calculate real-world MAE  
mae_real = np.mean(np.abs(y_pred_real - y_test_real))  
print(f"Real-World MAE: {mae_real:.2f} meters")  

Test MSE: 1.0107 (Normalized)
Test MAE: 0.8672 (Normalized)
47/47 [==============================] - 0s 3ms/step
Real-World MAE: 8222.47 meters
